In [1]:
fws = ['tflow','ptorch']

In [2]:
fw = fws[0]

In [3]:
fw

'tflow'

In [4]:
import organise_test as org

In [5]:
lenet1 = org.give_df('LeNet_1.{}'.format(fw))
lenet5 = org.give_df('LeNet_5.{}'.format(fw))
myalexnet = org.give_df('My_AlexNet.{}'.format(fw))
myvgg11 = org.give_df('My_VGG_11.{}'.format(fw))

In [6]:
import sys
sys.path.append('../')
import lib
if fw=='tflow':
    import tflow as tools
    import execute.tflow_models as tm
else:
    import ptorch as tools
    import execute.ptorch_models as tm

In [7]:
tools

<module 'tflow' from '/home/vlassis/Desktop/Diploma/gits/estimate/tflow.py'>

In [8]:
tm

<module 'execute.tflow_models' from '../execute/tflow_models.py'>

In [9]:
Regs = [None]+[lib.Reg_N(fw,node) for node in [1,2,3]]

In [10]:
class Pred:
    def __init__(self,Regs):
        self.Regs = Regs    
    
    def create(self,the_df,Model):
        def give(epochs,ds,numf,channels,batch,nodes):
            M = Model()
            model = M.create()
            layers = tools.extract(model, 2, batch, numf, channels)
            total = self.Regs[nodes].predict(layers, epochs, ds, batch)

            return total
        
        def pred(x):
            time = give(int(x['Epochs']),int(x['Dataset size']),int(x['Number of features']),
                        int(x['Channels']),int(x['Batch size']),int(x['Nodes']))
            return time
            
        the_df['Predicted(s)'] = the_df.apply(pred,axis=1)
        the_df['Difference(s)'] = abs(the_df['Predicted(s)']-the_df['Time (s)'])
        the_df['Error%'] = round(100*abs(the_df['Predicted(s)']-the_df['Time (s)'])/the_df['Time (s)'],1)
        
#         return the_df.drop(columns=['Epochs','Number of features', 'Channels'])
        return the_df.drop(columns=['Dataset size','Epochs','Number of features', 'Channels'])

In [11]:
def save(df,title):
    the = {
        'tflow':'TensorFlow',
        'ptorch':'PyTorch'
    }
    with open('tables/{}.tex'.format(fw), 'a') as file:
#         df.to_latex(file,caption='{} - {}'.format(the[fw],title), 
#                     label='fig:{}_{}'.format(fw,title),
#                     index=False)
        tab = df.to_latex(caption='{}-{}'.format(the[fw],title), 
                    label='fig:{}_{}'.format(fw,title),
                    index=False)

        [pa,pb] = tab.split('\\begin{tabular}',1)
        [pb1,pb2] = pb.split('\\end{tabular}',1)
        
        pa += '\\scalebox{0.8}{\n'
        pb2 = '\n}' + pb2
        
        res = pa+'\\begin{tabular}'+pb1+'\\end{tabular}'+pb2
        file.write(res+'\n\n')
        
    with open('tables/{}.csv'.format(fw), 'a') as file:
        df.to_csv(file,index=False)

In [12]:
def update(mean,var):
    the = {
        'tflow':'TensorFlow',
        'ptorch':'PyTorch'
    }
    with open('tables/{}.tex'.format(fw), 'a') as file:
        file.write('Μέσος όρος σφάλματος: {}\\% \n\nΤυπική απόκλιση σφάλματος: {}\\% \n\n'.format(mean, var))

In [13]:
p = Pred(Regs)

In [14]:
df_lenet1 = p.create(lenet1,tm.LeNet_1)

In [15]:
df_lenet1

,Batch size,Nodes,Time (s),Predicted(s),Difference(s),Error%
0,16.0,1.0,5.552860,6.111523,0.558663,10.1
1,16.0,2.0,2.776430,3.074646,0.298216,10.7
2,16.0,3.0,1.850953,2.039735,0.188782,10.2
3,32.0,1.0,5.129190,5.286196,0.157006,3.1
4,32.0,2.0,2.564595,2.638879,0.074285,2.9
5,32.0,3.0,1.709730,1.755035,0.045305,2.6
6,64.0,1.0,4.768607,4.873947,0.105340,2.2
7,64.0,2.0,2.384304,2.451200,0.066896,2.8
8,64.0,3.0,1.589536,1.628707,0.039171,2.5
9,128.0,1.0,3.726668,4.378842,0.652174,17.5


In [16]:
save(df_lenet1.head(12),'LeNet1 (4608)')

In [17]:
save(df_lenet1.tail(12),'LeNet1 (36864)')

In [18]:
df_lenet5 = p.create(lenet5,tm.LeNet_5)

In [19]:
df_lenet5

,Batch size,Nodes,Time (s),Predicted(s),Difference(s),Error%
0,16.0,1.0,7.766443,9.523006,1.756563,22.6
1,16.0,2.0,3.883221,4.691968,0.808747,20.8
2,16.0,3.0,2.588814,3.167750,0.578935,22.4
3,32.0,1.0,6.550888,7.453351,0.902463,13.8
4,32.0,2.0,3.275444,3.752042,0.476598,14.6
5,32.0,3.0,2.183629,2.515338,0.331709,15.2
6,64.0,1.0,6.107675,7.521091,1.413416,23.1
7,64.0,2.0,3.053838,3.661644,0.607807,19.9
8,64.0,3.0,2.035892,2.514241,0.478349,23.5
9,128.0,1.0,5.306523,7.839533,2.533011,47.7


In [20]:
save(df_lenet5.head(12),'LeNet5 (4608)')

In [21]:
save(df_lenet5.tail(12),'LeNet5 (36864)')

In [22]:
df_myvgg1 = p.create(myvgg11,tm.My_VGG_11)

In [23]:
df_myvgg1

,Batch size,Nodes,Time (s),Predicted(s),Difference(s),Error%
0,16.0,1.0,332.963549,160.402234,172.561316,51.8
1,16.0,2.0,166.481775,80.325337,86.156438,51.8
2,16.0,3.0,110.987850,53.485581,57.502269,51.8
3,32.0,1.0,304.234759,162.662979,141.571780,46.5
4,32.0,2.0,152.117379,81.400794,70.716585,46.5
5,32.0,3.0,101.411586,54.036608,47.374978,46.7
6,64.0,1.0,257.896816,149.891607,108.005209,41.9
7,64.0,2.0,128.948408,75.166832,53.781576,41.7
8,64.0,3.0,85.965605,49.863162,36.102444,42.0
9,128.0,1.0,238.213484,140.151042,98.062442,41.2


In [24]:
save(df_myvgg1,'VGG11 (4608)')

In [25]:
def to_vec(*args):
    vec = []
    for elem in args:
        vec += list(elem)
    return vec

In [26]:
errs = to_vec(df_lenet1['Error%'],df_lenet5['Error%'],df_myvgg1['Error%'])

In [27]:
import statistics as st

In [28]:
mean = st.mean(errs)

In [29]:
mean

24.231666666666666

In [30]:
var = st.stdev(errs)

In [31]:
var

17.247917994391177

In [32]:
update(round(mean,2),round(var,2))

In [33]:
# df_myalexnet = p.create(myalexnet,tm.My_AlexNet)

# df_myalexnet

# save(df_myalexnet,'AlexNet')